In [549]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader as pdr
import pandas._testing as tm
import numpy as np
import math

In [510]:
# В публикациях стала чаще упоминаться гипотеза, что ранее не было зависимости Биткоина от биржевых 
# индексов, но с течением времени эта корреляция становиться более заметной. Проверим данную гипотезу.
# Произведем расчет коэффициэнта корреляции с начала 2022 года.

data = yf.download("BTC-USD, GC=F, ^GSPC, ^IXIC, ^VIX", start="2022-01-02", end='2022-04-26')

[*********************100%***********************]  5 of 5 completed


In [511]:
df = data[['Volume', "Close"]] # Оставляем данные: Объем и дневная Цена закрытия
df1 = df.drop([('Volume','^VIX')], axis=1) # Убираем колонку "Объем" "Волатильность" т.к. отсутствуют данные

In [512]:
data[["Close"]]

Close                                                   
                 BTC-USD         GC=F        ^GSPC         ^IXIC       ^VIX
Date                                                                       
2022-01-01  47686.812500          NaN          NaN           NaN        NaN
2022-01-02  47345.218750          NaN          NaN           NaN        NaN
2022-01-03  46458.117188  1799.400024  4796.560059  15832.799805  16.600000
2022-01-04  45897.574219  1814.000000  4793.540039  15622.719727  16.910000
2022-01-05  43569.003906  1824.599976  4700.580078  15100.169922  19.730000
...                  ...          ...          ...           ...        ...
2022-04-21  40527.363281  1944.900024  4393.660156  13174.650391  22.680000
2022-04-22  39740.320312  1931.000000  4271.779785  12839.290039  28.209999
2022-04-23  39486.730469          NaN          NaN           NaN        NaN
2022-04-24  39469.292969          NaN          NaN           NaN        NaN
2022-04-25  40458.308594  1893.199951  4296.120117  13004.849609  27.020000

[115 rows x 5 columns]

In [513]:
df = data[([('Close', '^GSPC'), ('Close', 'BTC-USD')])]
df.columns = '^GSPC','BTC-USD' #
ind = df.index
# df1 = df.fillna(0, axis=0)
df

,^GSPC,BTC-USD
Date,,
2022-01-01,NaN,47686.812500
2022-01-02,NaN,47345.218750
2022-01-03,4796.560059,46458.117188
2022-01-04,4793.540039,45897.574219
2022-01-05,4700.580078,43569.003906
...,...,...
2022-04-21,4393.660156,40527.363281
2022-04-22,4271.779785,39740.320312
2022-04-23,NaN,39486.730469


In [514]:
# Подставим средние значения в неторговые дни на бирже. 
# Затем нужно убрать данную подстановку и посмотреть на изменение корреляции.
df1 = df.fillna(df.mean())
df1

,^GSPC,BTC-USD
Date,,
2022-01-01,4459.056015,47686.812500
2022-01-02,4459.056015,47345.218750
2022-01-03,4796.560059,46458.117188
2022-01-04,4793.540039,45897.574219
2022-01-05,4700.580078,43569.003906
...,...,...
2022-04-21,4393.660156,40527.363281
2022-04-22,4271.779785,39740.320312
2022-04-23,4459.056015,39486.730469


In [515]:
#Построим график
x_1 = df1['^GSPC']
y_1 = df1['BTC-USD'] 
import plotly.express as px
fig = px.scatter(x=x_1,y=y_1)
fig.show()
# Вопрос: что это за линия, которая выстроилась со значениями индекса S&P равными 4459.056? 
# Оказывается это наши подставленные средние значения. Думаю, это искусственные мои вбросы
# и от них нужно избавиться.

In [545]:
# df.loc[df['^GSPC']==0, ['BTC-USD']]=0 #Для тренировки
df2 = df.fillna(0, axis=0)
df2 = df2[df2['^GSPC']>0] 
# Исключили нулевые значения. Думаю можно принебречь 
# несколькими неторговыми днями (выходные и праздники).
df2

,^GSPC,BTC-USD
Date,,
2022-01-03,4796.560059,46458.117188
2022-01-04,4793.540039,45897.574219
2022-01-05,4700.580078,43569.003906
2022-01-06,4696.049805,43160.929688
2022-01-07,4677.029785,41557.902344
...,...,...
2022-04-19,4462.209961,41502.750000
2022-04-20,4459.450195,41374.378906
2022-04-21,4393.660156,40527.363281


In [568]:
# Произведем расчет простого коэффициента корреляции Пирсона на основании наших очищенных данных.
x = df2['^GSPC']
y = df2['BTC-USD'] 
cor_ch = (len(df2)*sum(x*y))-(sum(x)*sum(y))
cor_zn = math.sqrt(len(df2)*sum(x**2) - ((sum(x))**2))*math.sqrt((len(df2)*(sum(y**2)) - (sum(y))**2))
#cor_ch
cor_zn
n = (cor_ch/cor_zn)
n
#cor_ch

0.5985861779781696

In [518]:
import plotly.express as px
fig = px.scatter(x=x,y=y)
fig.show()

Промежуточные итоги: коэффициэнт корреляции 0.598. Попробуем изменить данные с дневных на недельные. Возможно это будет нагляднее.

In [519]:
df3=df2.reset_index()
df3

,Date,^GSPC,BTC-USD
0,2022-01-03,4796.560059,46458.117188
1,2022-01-04,4793.540039,45897.574219
2,2022-01-05,4700.580078,43569.003906
3,2022-01-06,4696.049805,43160.929688
4,2022-01-07,4677.029785,41557.902344
...,...,...,...
73,2022-04-19,4462.209961,41502.750000
74,2022-04-20,4459.450195,41374.378906
75,2022-04-21,4393.660156,40527.363281
76,2022-04-22,4271.779785,39740.320312


In [520]:
# Замысловатый способ найти среднее значение на недельном таймфрейме). Далее будет нормальный...
df3["month"] = df3["Date"].dt.isocalendar().week 
df3

,Date,^GSPC,BTC-USD,month
0,2022-01-03,4796.560059,46458.117188,1
1,2022-01-04,4793.540039,45897.574219,1
2,2022-01-05,4700.580078,43569.003906,1
3,2022-01-06,4696.049805,43160.929688,1
4,2022-01-07,4677.029785,41557.902344,1
...,...,...,...,...
73,2022-04-19,4462.209961,41502.750000,16
74,2022-04-20,4459.450195,41374.378906,16
75,2022-04-21,4393.660156,40527.363281,16
76,2022-04-22,4271.779785,39740.320312,16


In [521]:
snp_mean = []
btc_mean = []
db = pd.DataFrame()
for i in range(1,18):
    z=df3[df3['month']==i]
    snp_mean.append(z['^GSPC'].mean())
    btc_mean.append(z['BTC-USD'].mean())
db['S&P'] = snp_mean
db['BTC'] = btc_mean
db

,S&P,BTC
0,4732.751953,44128.705469
1,4686.317969,42839.497656
2,4497.634888,40314.423828
3,4374.974023,37076.603906
4,4525.887891,38566.971875
5,4503.062109,43654.115625
6,4415.375879,42338.593750
7,4300.902466,38282.356445
8,4351.820020,42612.276563
9,4222.700000,39402.933594


In [522]:

fig = px.scatter(x=db['BTC'],y=db['S&P'])
fig.show()


In [535]:
# Еще один способ сгруппировать недельные таймфреймы и найти среднее значение. Так проще.
df3.drop("month", axis=1, inplace= True)
week_1 = df3.groupby(df3["Date"].dt.isocalendar().week).mean()
week_1

,^GSPC,BTC-USD
week,,
1,4732.751953,44128.705469
2,4686.317969,42839.497656
3,4497.634888,40314.423828
4,4374.974023,37076.603906
5,4525.887891,38566.971875
6,4503.062109,43654.115625
7,4415.375879,42338.593750
8,4300.902466,38282.356445
9,4351.820020,42612.276563


In [550]:
# Рсчет коэффициента корреляции.
week_1["^GSPC"].corr(week_1["BTC-USD"])

0.6019371330246444

In [536]:
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from pyecharts.charts import Bar
from pyecharts import options as opts
from datetime import datetime
from datetime import timedelta
import pandas_datareader as data_reader
stocktickers = ['GOOG', 'AAPL', 'FB']
stock = []

#Извлечение данных за 14 дней
start = (datetime.now()-timedelta(days=14)).strftime('%Y-%m-%d')
end = (datetime.now()).strftime('%Y-%m-%d')

for stockticker in stocktickers:
    data = data_reader.DataReader(stockticker, 'yahoo', start, end)
    data['stock_ticker'] = stockticker
    stock.append(data)

stockDF = pd.concat(stock)

date = stockDF.index.unique().tolist()
datedata = [x.strftime("%Y-%m-%d") for x in date]
GOOG = stockDF[stockDF['stock_ticker']=='GOOG']['Volume'].tolist()
AAPL = stockDF[stockDF['stock_ticker']=='AAPL']['Volume'].tolist()
FB = stockDF[stockDF['stock_ticker']=='FB']['Volume'].tolist()

bar = (Bar()
       .add_xaxis(date)
       .add_yaxis('GOOG', GOOG)
       .add_yaxis('AAPL', AAPL)
       .add_yaxis('FB', FB)
       .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max", name="Maximum "),
                                                                opts.MarkPointItem(type_="min", name="Minimum")]))
       .set_global_opts(title_opts=opts.TitleOpts(title="Stock Market", subtitle="Daily Volume"),
                                       datazoom_opts=opts.DataZoomOpts(range_start=0, range_end=100),
                                    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
)
       )
bar.render_notebook()